In [31]:
import numpy as np
import functools


In [32]:


# |0> = [1,0], |1> = [0,1]
H = np.array([[1,1],[1,-1]])/np.sqrt(2)

def phaseEstimation(t,U,e): 
    """
    Performs phase estimation.

    Inputs:
    t - accuracy of psi returned
    U - unitary matrix
    e - normalised ket
    """
    N = U.shape[0]

    @functools.lru_cache(maxsize=N)
    def pow(n,prev=np.identity(N)):
        next = U @ prev
        if n == 1:
            return next
        return pow(U,n-1,next)

    f = np.zeros([t,2])

    for i in range(t):
        s = np.array([1,0])
        s = H @ s

        #apply 
        s[1] *= e @ pow(2**i) @ e
        f[t-1-i] = s

    return f #apply inverse QFT


In [100]:
def invQFT(f):
    """
    Performs inverse quantum fourier transform on a tensor products of qubit states.

    Intputs:
    f - tensor product of qubit states, in form [[qubit 1 state], [qubit 2 state], ...]
    """

    N = f.shape[0]
    fInv = np.empty([N,2],dtype=np.complex_)

    for i in range(N): 
        s = f[i]
        for k in range(2,N+1-i):
            s[1] *= np.exp(-2*np.pi*complex(0,1)/(2**k))
        s = H @ s
        fInv[i] = s
    
    return fInv


In [101]:
def findNumber(s):
    """
    Function that finds the associated number for a qubit tensor product state. 
    """
    binaryForm = np.empty(s.shape[0])
    for i,ket in enumerate(s):
        if np.allclose(ket,np.array([1,0])):
            binaryForm[i] = 0
        elif np.allclose(ket,np.array([0,1])):
            binaryForm[i] = 1
        else:
            print("error")
    
    number = sum([b*2**i for i,b in enumerate(np.flip(binaryForm))])
    return number

        

In [102]:
#test case for invQFT: 3 qubit system, |2> = |010> state should be the result
invf = np.array([[1/np.sqrt(2),complex(-1,1)/2],[1/np.sqrt(2),-complex(0,1)/np.sqrt(2)],[1/np.sqrt(2),1/np.sqrt(2)]]) 
print(findNumber(invQFT(invf)))

2.0
